In [1]:
from estnltk import Text

from pandas import read_csv
from estnltk.taggers import RegexTagger

from estnltk.spans import Span, SpanList
from estnltk.layer import Layer
from estnltk.finite_grammar.layer_graph import print_nodes
import csv
from estnltk.finite_grammar import PhraseListTagger

from estnltk.taggers.gaps_tagging.gaps_tagger import GapsTagger
from estnltk.taggers.merge_tagging.merge_tagger import MergeTagger

In [2]:
example_sents = [
    'paremal kopsus väikesed tihedamad kolded',
  'paremas põlveliigeses ealised muutused',
  'kops nähtavas osas patoloogiata',
  'kopsud patoloogilise leiuta',
  'vasemas neerus tihe kolle',
]

In [3]:
def decorator(text, span, phrase):
    return {'match': phrase, 'grammar_symbol': 'TEST'}

In [4]:
bodyparts = [('kops',), 
            ('põlv',),
            ('neer',)]

In [5]:
test_tagger = PhraseListTagger(layer_name='test',
                               input_layer = 'morph_analysis',
                               input_attribute='lemma',
                               phrase_list=bodyparts,
                               decorator = decorator,
                               attributes=('grammar_symbol',),
                               conflict_resolving_strategy= 'MAX'
                              )

In [6]:
j = Text('paremal kopsus väikesed tihedamad kolded')

In [7]:
j.analyse('morphology')

Text(text="paremal kopsus väikesed tihedamad kolded")

In [8]:
from estnltk.layer_operations.flatten import flatten

test_layer = test_tagger.tag(j, return_layer=True)
j['test'] = flatten(test_layer, 'test')

In [9]:
def gaps_decorator(text:str):
    return {'grammar_symbol': 'RANDOM_TEXT'}

In [10]:
gaps_tagger = GapsTagger(layer_name='gaps',
                         input_layers=['test'],
                         #trim=trim, 
                         decorator=gaps_decorator,
                         attributes=['grammar_symbol'])

In [11]:
gaps_tagger.tag(j)

In [12]:
j

Text(text="paremal kopsus väikesed tihedamad kolded")

In [13]:
j.gaps

SL[Span(paremal , {'grammar_symbol': 'RANDOM_TEXT'}),
Span( väikesed tihedamad kolded, {'grammar_symbol': 'RANDOM_TEXT'})]

In [14]:
j.test

SL[Span(kopsus, {'grammar_symbol': 'TEST'})]

In [15]:
merge_tagger = MergeTagger(layer_name='grammar_tags',
                           input_layers=['test', 'gaps'],
                           attributes=('grammar_symbol', 'value'))

In [16]:
merge_tagger.tag(j)

In [17]:
j.grammar_tags

SL[Span(paremal , {'value': None, 'grammar_symbol': 'RANDOM_TEXT'}),
Span(kopsus, {'value': None, 'grammar_symbol': 'TEST'}),
Span( väikesed tihedamad kolded, {'value': None, 'grammar_symbol': 'RANDOM_TEXT'})]

In [18]:
j

Text(text="paremal kopsus väikesed tihedamad kolded")